In [ ]:
import numpy as np
import pandas as pd

#precisao de casas decimais
pd.set_option("display.precision", 3)
#maximo de colunas
pd.set_option("display.max_columns",200)

In [ ]:
df = pd.read_csv('./incident_event_log.csv', low_memory=False)
df.head()